Global Constants

In [1]:
import codecs, re, sys

sys.path.append("~/Documents/GitHub/Voices-Now-SRT-Generator/Scripts")

from Timecodes.Timecode import Timecode, OffsetType

from Captions.CaptionsWriter import CaptionMaker
from Languages.LanguageDatabase import LanguageDatabase
from Languages.Language import Language

from Projects.SRTProject import SRTProject, SRT_FOLDER_NAME, SRT_EXTENSION
from Projects.SRTOffset import SRTOffset

LANGUAGE_MANAGER = LanguageDatabase()

PROJECT_PATH = "MFD Training Videos for Coordinating Councils/Scripts/Translated Scripts"

TIMECODE_OFFSET = Timecode.from_string("00:00:14;25")
TIMECODE_OFFSET_TYPE = OffsetType.DELAY


def split_languages(languages: list[Language] = []):
    for language in languages:
        create_combined_srt(language)


def create_combined_srt(language: Language):
    project = SRTProject()

    project.set_language(language)
    project.set_project_path(PROJECT_PATH)

    combined_srt = project.format_file_path(language.name, SRT_FOLDER_NAME, "MFD_TrainingCouncilCombined", SRT_EXTENSION)

    with codecs.open(combined_srt, "w+", encoding="utf-8") as combined_file:
        id_offset = 0

        for i in range(1, 4):
            id_offset = create_single_srt(f"MFD_Training0{i}", id_offset)
            srt_name = project.format_file_path(language.name, SRT_FOLDER_NAME, f"MFD_Training0{i}", SRT_EXTENSION)

            with codecs.open(srt_name, "r", encoding="utf-8") as srt_file:
                combined_file.write(srt_file.read())


def create_single_srt(filename: str, id_offset: int):
    print(filename)

    project = SRTProject()
    project.open_script_from_filename(filename)
    project.open_session_from_filename(filename)
    
    offset = SRTOffset(TIMECODE_OFFSET, TIMECODE_OFFSET_TYPE, id_offset)
    project.set_srt_offset(offset)

    srt_maker = CaptionMaker()
    id_offset = srt_maker.create_captions()

    return id_offset

/Users/studiod/Documents/GitHub/Voices-Now-SRT-Generator/.venv/lib/python3.9/site-packages/urllib3/__init__.py:35: NotOpenSSLWarning: urllib3 v2 only supports OpenSSL 1.1.1+, currently the 'ssl' module is compiled with 'LibreSSL 2.8.3'. See: https://github.com/urllib3/urllib3/issues/3020
  warnings.warn(


ModuleNotFoundError: No module named 'Projects.Project'

In [2]:
codes = ["FRA"]

languages = [LANGUAGE_MANAGER.get_language(language) for language in codes]

split_languages(languages)

DEBUG:root:EDL: 1		01:00:25;18
DEBUG:root:EDL: 2		01:00:31;21
DEBUG:root:EDL: 3		01:00:36;10
DEBUG:root:EDL: 4		01:00:41;21
DEBUG:root:EDL: 5		01:00:48;25
DEBUG:root:EDL: 6		01:00:54;12
DEBUG:root:EDL: 7		01:01:02;19
DEBUG:root:EDL: 8		01:01:08;25
DEBUG:root:EDL: 9		01:01:13;01
DEBUG:root:EDL: 10		01:01:20;28
DEBUG:root:EDL: 11		01:01:27;24
DEBUG:root:EDL: 12		01:01:38;07
DEBUG:root:EDL: 13		01:01:45;13
DEBUG:root:EDL: 14		01:02:01;08
DEBUG:root:EDL: 15		01:02:11;17
DEBUG:root:EDL: 16		01:02:19;17
DEBUG:root:EDL: 17		01:02:25;00
DEBUG:root:EDL: 18		01:02:34;03
DEBUG:root:EDL: 19		01:02:40;09
DEBUG:root:EDL: 20		01:02:45;21
DEBUG:root:EDL: 21		01:02:49;19
DEBUG:root:EDL: 22		01:03:01;20
DEBUG:root:EDL: 23		01:03:14;04
DEBUG:root:EDL: 24		01:03:20;00
DEBUG:root:EDL: 25		01:03:28;17
DEBUG:root:EDL: 26		01:03:36;28
DEBUG:root:EDL: 27		01:03:44;08
DEBUG:root:EDL: 28		01:03:55;12
DEBUG:root:EDL: 29		01:04:04;15
DEBUG:root:EDL: 30		01:04:11;16
DEBUG:root:EDL: 31		01:04:27;23


/Volumes/VONGeneral/LDS/~Current Projects/MFD Training Videos for Coordinating Councils/Scripts/Translated Scripts/Indonesian/Scripts/MFD_TrainingCouncil01.docx


DEBUG:root:EDL: 1		01:04:43;22
DEBUG:root:EDL: 2		01:04:49;24
DEBUG:root:EDL: 3		01:04:56;00
DEBUG:root:EDL: 4		01:05:02;11
DEBUG:root:EDL: 5		01:05:15;07
DEBUG:root:EDL: 6		01:05:25;16
DEBUG:root:EDL: 7		01:05:31;18
DEBUG:root:EDL: 8		01:05:34;24
DEBUG:root:EDL: 9		01:05:44;17
DEBUG:root:EDL: 10		01:05:57;00
DEBUG:root:EDL: 11		01:06:04;29
DEBUG:root:EDL: 12		01:06:13;29
DEBUG:root:EDL: 13		01:06:25;23
DEBUG:root:EDL: 14		01:06:32;02
DEBUG:root:EDL: 15		01:06:39;12
DEBUG:root:EDL: 16		01:06:47;06
DEBUG:root:EDL: 17		01:06:54;27
DEBUG:root:EDL: 18		01:06:59;08
DEBUG:root:EDL: 19		01:07:08;15
DEBUG:root:EDL: 20		01:07:14;18
DEBUG:root:EDL: 21		01:07:18;08
DEBUG:root:EDL: 22		01:07:24;19
DEBUG:root:EDL: 23		01:07:28;21
DEBUG:root:EDL: 24		01:07:34;27
DEBUG:root:EDL: 25		01:07:44;22
DEBUG:root:EDL: 26		01:07:50;25
DEBUG:root:EDL: 27		01:07:55;09
DEBUG:root:EDL: 28		01:08:05;04


/Volumes/VONGeneral/LDS/~Current Projects/MFD Training Videos for Coordinating Councils/Scripts/Translated Scripts/Indonesian/Scripts/MFD_TrainingCouncil02.docx


DEBUG:root:EDL: 1		01:08:25;10
DEBUG:root:EDL: 2		01:08:29;13
DEBUG:root:EDL: 3		01:08:37;13
DEBUG:root:EDL: 4		01:08:41;16
DEBUG:root:EDL: 5		01:08:50;07
DEBUG:root:EDL: 6		01:09:06;19
DEBUG:root:EDL: 7		01:09:17;02
DEBUG:root:EDL: 8		01:09:23;09
DEBUG:root:EDL: 9		01:09:30;26
DEBUG:root:EDL: 10		01:09:45;06
DEBUG:root:EDL: 11		01:09:51;26
DEBUG:root:EDL: 12		01:09:58;20
DEBUG:root:EDL: 13		01:10:06;04
DEBUG:root:EDL: 14		01:10:13;21
DEBUG:root:EDL: 15		01:10:20;16
DEBUG:root:EDL: 16		01:10:27;24
DEBUG:root:EDL: 17		01:10:35;15
DEBUG:root:EDL: 18		01:10:39;29
DEBUG:root:EDL: 19		01:10:47;19
DEBUG:root:EDL: 20		01:10:53;23
DEBUG:root:EDL: 21		01:11:00;22
DEBUG:root:EDL: 22		01:11:06;04
DEBUG:root:EDL: 23		01:11:12;19
DEBUG:root:EDL: 24		01:11:19;28
DEBUG:root:EDL: 25		01:11:28;12
DEBUG:root:EDL: 26		01:11:32;24
DEBUG:root:EDL: 27		01:11:39;12
DEBUG:root:EDL: 28		01:11:45;29
DEBUG:root:EDL: 29		01:11:51;21


/Volumes/VONGeneral/LDS/~Current Projects/MFD Training Videos for Coordinating Councils/Scripts/Translated Scripts/Indonesian/Scripts/MFD_TrainingCouncil03.docx
